# This is a Pytorch Loghtning starter code
I have trained efficinet-net b2 for 6 epochs and the lb score is .93
# Room for improvement 
* Add effective augmentation
* Train larger models from timm models for longer epochs

Referance notebooks

[Simple yet EffectiveB0](https://www.kaggle.com/micheomaano/simple-yet-effectiveb0-lb-0-96)

[⚡Plant2021 PyTorch Lightning Starter [ Training ]⚡](https://www.kaggle.com/pegasos/plant2021-pytorch-lightning-starter-training)


Feel free to give this notebook an upvote if it is helpful!!

# Imports 

In [ ]:
!pip install -q iterative-stratification
import sys; 
sys.path.insert(0,'../input/timm-all-models/pytorch-image-models-master/pytorch-image-models-master')
# sys.path.insert(0, '../input/timm-all-models')

import torchmetrics
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import albumentations as A
import cv2
from tqdm.notebook import tqdm



from albumentations.pytorch import ToTensorV2

import timm
from timm.utils.agc import adaptive_clip_grad

import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast, GradScaler

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from albumentations.core.composition import Compose, OneOf
warnings.simplefilter("ignore")
from pytorch_lightning import Trainer, seed_everything
import torch.nn.functional as F


from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn import metrics
import itertools

# Config
Fell free to change the hyperparameters

In [ ]:
class Config:
    seed=42
    n_epoch=30
    img_size= 520
    train_path='../input/plant-pathology-2021-fgvc8/train_images'
    test_path='../input/plant-pathology-2021-fgvc8/test_images'
    lr = 1e-4
    max_lr = 1e-3
    pct_start = 0.3
    div_factor = 1.0e+3
    final_div_factor = 1.0e+3
    accum = 1
    precision = 16
    n_fold = 3
    num_classes=1
    
    
    
    weight_decay=0.001
    debug=False
    debug_sample=100
    train_batch=32
    test_batch=32
    path='../input/plant-pathology-2021-fgvc8/'
    
        
device = torch.device("cuda")

seed_everything(Config.seed)

# Loading the data and adding the image_path

In [ ]:
df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
print (df.shape)
df['img_path'] = df['id'].apply(lambda x: f'../input/seti-breakthrough-listen/train/{x[0]}/{x}.npy')

# Dataset

In [ ]:
class SetiDataset:
    
    def __init__(self, df): 
        self.image_paths = df['img_path']
        self.targets = df['target']

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):      
        image = np.load(self.image_paths[idx]).astype(float)

        targets = [self.targets[idx]]
                
        return {
            "image": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.float),
        }

# MODEL
I have chosen efficientnet b2 from timm models
You can choose any model from  timm.list_models(pretrained=True)

In [ ]:
class Efnetmodel(nn.Module):
    
    def __init__(self, model_name='efficientnet_b2', pretrained=True):
        super().__init__()
        self.conv1 = nn.Conv2d(6, 3, kernel_size=3, stride=1, padding=3, bias=False)
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
#         self.model.fc = nn.Linear(in_features, CFG.num_classes)
        self.model.classifier = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, Config.num_classes)
        )
        

    def forward(self, x):
        x=self.conv1(x)
        
        x = self.model(x)
       
    
        
        return x
    def save(self,optim,epoch):
        self.eval()
        torch.save({
            'epoch': epoch,
            'model_state_dict': self.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': 0,
            }, './effnet_new_b2{}.pth'.format(epoch))
    def load(self,optim,path):
        checkpoint = torch.load(path)
        self.load_state_dict(checkpoint['model_state_dict'])
        optim.load_state_dict(checkpoint['optimizer_state_dict'])
        self.epoch = checkpoint['epoch']
        self.loss = checkpoint['loss']

In [ ]:
folds = df.copy()
Fold = StratifiedKFold(n_splits=Config.n_fold, shuffle=True, random_state=Config.seed)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds['target'])):
    df_train = df.iloc[train_index]
    df_valid = df.iloc[val_index]
df_train=df_train.reset_index(drop=True)
df_valid=df_valid.reset_index(drop=True)

In [ ]:
train_dataset=SetiDataset(df_train)
valid_dataset=SetiDataset(df_valid)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16,shuffle=True, num_workers=4)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=16,shuffle=False, num_workers=4)

In [ ]:
Config.steps_per_epoch = len(train_loader)
Config.steps_per_epoch

In [ ]:
class LitPlant(pl.LightningModule):
    def __init__(self, model):
        super(LitPlant, self).__init__()
        self.model = model
#         self.metric = pl.metrics.F1(num_classes=CFG.num_classes)
#         self.metric = torchmetrics.F1(Config.num_classes, average='weighted')
#         self.criterion = nn.BCELoss()
        self.criterion = nn.BCEWithLogitsLoss()
#         self.criterion = FocalLoss()
        self.sigmoid = nn.Sigmoid()
        self.lr = Config.lr

    def forward(self, x, *args, **kwargs):
        return self.model(x)

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
#         self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=CFG.t_max, eta_min=CFG.min_lr)
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(self.optimizer, 
                                                             epochs=Config.n_epoch, steps_per_epoch=Config.steps_per_epoch,
                                                             max_lr=Config.max_lr, pct_start=Config.pct_start, 
                                                             div_factor=Config.div_factor, final_div_factor=Config.final_div_factor)
        scheduler = {'scheduler': self.scheduler, 'interval': 'step',}

        return [self.optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        image = batch['image']
        target = batch['targets']
        output = self.model(image)
#         output = self.sigmoid(output)
        loss = self.criterion(output, target)
#         score = self.metric(self.sigmoid(output), target.clone().detach().to(torch.int32))
        pred=self.sigmoid(output).clone().detach().tolist()
        target=target.clone().detach().tolist()
        
        
        
        
            
        
        
        return {'loss':loss,'preds': pred ,'targets':target}
    def training_epoch_end(self, training_step_outputs):
        
        loss_list=[]
        pred_list=[]
        target_list=[]
        for out in training_step_outputs:
            loss_list.extend([out['loss'].cpu().detach().tolist()])
            pred_list.extend(out['preds'])
            target_list.extend(out['targets'])
            
            
        auc_score=metrics.roc_auc_score( target_list,pred_list   )
        meanloss=sum(loss_list )/len(loss_list )
        
        logs = {'train_loss': meanloss, 'train_auc': auc_score, 'lr': self.optimizer.param_groups[0]['lr']}
        
        self.log_dict(
            logs,
            on_step=False, on_epoch=True, prog_bar=True, logger=True)
        
            
            
            
            
            
            
            
            
    
    def validation_step(self, batch, batch_idx):
        image = batch['image']
        target = batch['targets']
        output = self.model(image)
#         output = self.sigmoid(output)
        loss = self.criterion(output, target)
#         score = self.metric(self.sigmoid(output), target.clone().detach().to(torch.int32))
        
        
#         valid_auc=  metrics.roc_auc_score(target.clone().detach().tolist(),self.sigmoid(output).clone().detach().tolist())
        pred=self.sigmoid(output).clone().detach().tolist()
        target=target.clone().detach().tolist()
        
        return {'loss':loss,'preds': pred ,'targets':target}
    
    
    def validation_epoch_end(self, validation_step_outputs):
        loss_list=[]
        pred_list=[]
        target_list=[]
        for out in validation_step_outputs:
            loss_list.extend([out['loss'].cpu().detach().tolist()])
            pred_list.extend(out['preds'])
            target_list.extend(out['targets'])
            
            
        auc_score=metrics.roc_auc_score( target_list,pred_list   )
        meanloss=sum(loss_list )/len(loss_list )
        
        logs = {'valid_loss': meanloss, 'valid_auc': auc_score, }
        
        self.log_dict(
            logs,
            on_step=False, on_epoch=True, prog_bar=True, logger=True)
            
       

In [ ]:

model =  Efnetmodel()
lit_model = LitPlant(model)

In [ ]:
checkpoint_callback = ModelCheckpoint(monitor='valid_auc',
                                      save_top_k=1,
                                      save_last=True,
                                      save_weights_only=True,
                                      filename='{epoch:02d}-{valid_loss:.4f}-{valid_auc:.4f}',
                                      verbose=False,
                                      mode='max',
                                      dirpath='./')

trainer = Trainer(
    max_epochs=Config.n_epoch,
    gpus=[0],
    accumulate_grad_batches=Config.accum,
    precision=Config.precision,
#     callbacks=[EarlyStopping(monitor='valid_loss', patience=3, mode='min')],
    checkpoint_callback=checkpoint_callback,
#     logger=logger,
    weights_summary='top',
    amp_backend='native',
    auto_lr_find=True 
)

In [ ]:
trainer.fit(lit_model, train_dataloader=train_loader, val_dataloaders=valid_loader)

In [ ]:
# saving the model the old fasion way though pytorch lightning does this for you feel free to comment out

model.save(lit_model.optimizer,Config.n_epoch)

# Infarence

In [ ]:
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            inputs = data["image"]
            targets = data["targets"]
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            output = model(inputs)
            
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
            
    return final_outputs, final_targets

# Run the cell below if you want to load weights otherwise comment it out
I have trained effcientnet b2 for 6 epochs and the result is quite good

In [ ]:
# model=Efnetmodel()
# optim=torch.optim.Adam(model.parameters(), lr=Config.lr)
# model.load(optim,'../input/seti-model-cheakpoint/seti_efficientnetb2e6/effnet_new_b26.pth')
# model=model.to(device)

In [ ]:
submission = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
submission['img_path'] = submission['id'].apply(lambda x: f'../input/seti-breakthrough-listen/test/{x[0]}/{x}.npy')

In [ ]:
test_dataset = SetiDataset(submission)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [ ]:
model=model.to(device)
predictions, valid_targets = evaluate(test_loader, model, device=device)

In [ ]:
predictions = np.array(predictions)[:, 0]
sig = torch.nn.Sigmoid()
outs = sig(torch.from_numpy(predictions))
outs = outs.detach().numpy()

submission.target = outs
submission.drop(['img_path'], axis=1, inplace=True)
submission.to_csv('submission.csv', index=False)